## derivatives

In [34]:
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [17]:
# analytical gradient
def gradf(a, b, c):
  # [df/da, df/db, df/dc]
  first = -3*a**2 - 0.5*a**-0.5
  second = cos(3*b) * b + 2.5*b**1.5
  third = 1*c**-2
  return [first, second, third]


ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [18]:
# estimate the gradient numerically
h = 0.0000001
a = 2
b = 3
c = 4
grad_a = (f(a+h, b, c) - f(a, b, c)) / h
grad_b = (f(a, b+h, c) - f(a, b, c)) / h
grad_c = (f(a, b, c+h) - f(a, b, c)) / h

# -----------
numerical_grad = [grad_a, grad_b, grad_c]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553966448771
OK for dim 1: expected 10.25699027111255, yours returns 10.256990385926201
OK for dim 2: expected 0.0625, yours returns 0.06249999628948899


In [29]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# https://en.wikipedia.org/wiki/Symmetric_derivative
h = 0.0000001
a = 2.0
b = 3.0
c = 4.0
grad2_a = (f(a+h, b, c) - f(a-h, b, c)) / (2*h)
grad2_b = (f(a, b+h, c) - f(a, b-h, c)) / (2*h)
grad2_c = (f(a, b, c+h) - f(a, b, c-h)) / (2*h)

# -----------
numerical_grad2 = [grad2_a, grad2_b, grad2_c]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990252699438
OK for dim 2: expected 0.0625, yours returns 0.06250000073038109


## support for softmax

In [82]:
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

    
  def __repr__(self):
    return f"Value(data={self.data})"

  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), label="*")
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad  
    out._backward = _backward
    
    return out
  
  
  def __radd__(self, other):
    return self + other
  
  
  def exp(self):
    x = exp(self.data)
    out = Value(x, (self, ), label=f'e**{x}')
    
    def _backward():
      self.grad += x * out.grad
    out._backward = _backward
    
    return out
  
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), 'int or float expeted for __pow__'
    out = Value(self.data**other, (self, ), label=f'{self.data}**{other}')
    def _backward():
      self.grad +=  other * (self.data**(other-1)) * out.grad
    out._backward = _backward
    return out
  
  
  def __truediv__(self, other):
    return self * (other**-1)
  
  
  def log(self):
    out = Value(log(self.data), (self, ), label='log')
    def _backward():
      self.grad += (self.data**-1) * out.grad
    out._backward = _backward
    return out
    
  def __neg__(self):
    return self * -1
 

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [81]:
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(probs)
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [71]:
print(log(0.11354961935990122))
print(math.e ** -2.1755153627039117)


-2.1755153626167147
0.11354961935000005


In [79]:
# verify the gradient using the torch
import torch

x1 = torch.Tensor([0.0]).double() ; x1.requires_grad = True
x2 = torch.Tensor([3.0]).double() ; x2.requires_grad = True
x3 = torch.Tensor([-2.0]).double() ; x3.requires_grad = True
x4 = torch.Tensor([1.0]).double() ; x4.requires_grad = True
logits = [x1, x2, x3, x4]
probs = softmax(logits)
loss = -probs[3].log()
print(probs)
print(loss.item())
loss.backward()

ns = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

[tensor([0.0418], dtype=torch.float64, grad_fn=<DivBackward0>), tensor([0.8390], dtype=torch.float64, grad_fn=<DivBackward0>), tensor([0.0057], dtype=torch.float64, grad_fn=<DivBackward0>), tensor([0.1135], dtype=torch.float64, grad_fn=<DivBackward0>)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns tensor([0.0418], dtype=torch.float64)
OK for dim 1: expected 0.8390245074625319, yours returns tensor([0.8390], dtype=torch.float64)
OK for dim 2: expected 0.005653302662216329, yours returns tensor([0.0057], dtype=torch.float64)
OK for dim 3: expected -0.8864503806400986, yours returns tensor([-0.8865], dtype=torch.float64)
